In [129]:
import pandas as pd
import geopandas
import datetime
from geopandas import GeoDataFrame
from shapely.geometry import Point
import matplotlib.pyplot as plt
from shapely.ops import nearest_points
from math import sin, cos, sqrt, atan2, radians
import math
from numpy import cos, sin, arcsin, sqrt
from IPython.display import display

In [130]:
#import fire calls into pandas
fireCalls = pd.read_csv("Seattle_Real_Time_Fire_911_Calls.csv")

#remove unnecessary initial line
fireCalls = fireCalls.drop([0, 0])

#add columns to break down data further
#https://erikrood.com/Python_References/extract_month_year_pandas_final.html
fireCalls['Month'] = pd.DatetimeIndex(fireCalls['Datetime']).month
fireCalls['Year'] = pd.DatetimeIndex(fireCalls['Datetime']).year

In [131]:
#separate 2017 and 2018 record
fc2017 = fireCalls[fireCalls.Year == 2017]
fc2018 = fireCalls[fireCalls.Year == 2018]

In [132]:
#import seattle neighborhood shapefile
seaNH = geopandas.read_file("City_Clerk_Neighborhoods\City_Clerk_Neighborhoods.shp")

#need to remove excess rows at beginning for extra islands
seaNH = seaNH.drop(seaNH.index[:26])

#get seaNH projection data to use for fire calls geodata frame 
seaNH_crs = seaNH.crs

In [133]:
#convert fireCalls to geodataframe
#https://gis.stackexchange.com/questions/174159/convert-a-pandas-dataframe-to-a-geodataframe
geometry1 = [Point(xy) for xy in zip(fc2017.Longitude, fc2017.Latitude)]
crs = seaNH_crs
fc2017_geo = GeoDataFrame(fc2017, crs = crs, geometry = geometry1)

geometry2 = [Point(xy) for xy in zip(fc2018.Longitude, fc2018.Latitude)]
crs = seaNH_crs
fc2018_geo = GeoDataFrame(fc2018, crs = crs, geometry = geometry2)


In [134]:
#spatial join fc_geo to neighborhoods so each fire call has a neighborhood attribute
fc2017_seattle = geopandas.sjoin(fc2017_geo, seaNH, how = 'inner', op = 'within')
fc2018_seattle = geopandas.sjoin(fc2018_geo, seaNH, how = 'inner', op = 'within')

D:\Anaconda\envs\mapping\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in find_contains (vectorized)
  outputs = ufunc(*inputs)


In [135]:
fc2017_seattle.to_file("2017_Fire_Calls_Seattle")
fc2018_seattle.to_file("2018_Fire_Calls_Seattle")

In [166]:
fc2017 = fc2017_seattle
fc2018 = fc2018_seattle

In [167]:
#load in fire stations and hospital locations
fireS = pd.read_csv("Fire_Stations.csv")
hospital = pd.read_csv("Hospitals.csv")

In [168]:
#convert to geodataframes
fs_geom = [Point(xy) for xy in zip(fireS.X, fireS.Y)]
crs = seaNH_crs
fs_geo = GeoDataFrame(fireS, crs = crs, geometry = fs_geom)

In [169]:
hos_geom = [Point(xy) for xy in zip(hospital.X, hospital.Y)]
crs = seaNH_crs
hospital_geo = GeoDataFrame(hospital, crs = crs, geometry = hos_geom)

In [170]:
#nearest neighbor analysis
#https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe
pts3 = fs_geo.geometry.unary_union
def near(point, pts=pts3):
    nearest = fs_geo.geometry == nearest_points(point, pts)[1]
    return fs_geo[nearest].STNID.get_values()[0]
fc2017['STNID'] = fc2017.apply(lambda row: near(row.geometry), axis=1)
fc2018['STNID'] = fc2018.apply(lambda row: near(row.geometry), axis=1)

In [176]:
hospital_geo= hospital_geo.rename(index=str, columns={"OBJECTID": "HOSID"})
pts3 = hospital_geo.geometry.unary_union
def near2(point, pts=pts3):
    nearest = hospital_geo.geometry == nearest_points(point, pts)[1]
    return hospital_geo[nearest].HOSID.get_values()[0]
fc2017['HOSID'] = fc2017.apply(lambda row: near2(row.geometry), axis=1)
fc2018['HOSID'] = fc2018.apply(lambda row: near2(row.geometry), axis=1)
fc2017 = fc2017.drop(columns=['Nearest_Fire_Station'])
fc2018 = fc2018.drop(columns=['Nearest_Fire_Station'])

In [182]:
fc17test = fc2017

In [183]:
fc18test = fc2018

In [184]:
#remove extra columns
fireS = fireS.drop(columns=['AREA', 'PERIMETER', 'FIRE_', 'FIRE_ID', 'B1', 'STREETS_', 'STREETS_ID', 'STREETS_SI', 'CIP', 'SE_ANNO_CAD_DATA', 'NAME1'])
hospital = hospital.drop(columns=['SE_ANNO_CAD_DATA', 'ACUTE_CARE', 'GIS_EDT_DT'])

In [185]:
#join closest firestation or hospital attributes to call data
fc17test = fc17test.merge(fireS, on="STNID")
fc17test = fc17test.merge(hospital, on="HOSID")

In [186]:
fc18test = fc18test.merge(fireS, on="STNID")
fc18test = fc18test.merge(hospital, on="HOSID")

In [187]:
#calculate the distance between each point
#https://stackoverflow.com/questions/25767596/vectorised-haversine-formula-with-a-pandas-dataframe
def calculate_distance(row):
    lon1 = row['Longitude']
    lat1 = row['Latitude']
    lon2 = row['X_x']
    lat2 = row['Y_x']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return km

fc17test['FS_Distance'] = fc17test.apply(lambda row: calculate_distance(row), axis=1)

In [188]:
fc18test['FS_Distance'] = fc18test.apply(lambda row: calculate_distance(row), axis=1)

In [189]:
def calculate_distance2(row):
    lon1 = row['Longitude']
    lat1 = row['Latitude']
    lon2 = row['X_y']
    lat2 = row['Y_y']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return km

fc17test['Hos_Distance'] = fc17test.apply(lambda row: calculate_distance2(row), axis=1)
fc18test['Hos_Distance'] = fc18test.apply(lambda row: calculate_distance2(row), axis=1)

In [197]:
fc2017 = fc17test
fc2018 = fc18test

#drop geometry rows for exporting to shapefile
fc2017 = fc2017.drop(columns=['geometry_x', 'geometry_y', 'geometry'])
fc2018 = fc2018.drop(columns=['geometry_x', 'geometry_y', 'geometry'])



In [200]:
pd.options.display.max_columns = None
#convert final dataframe to geodataframe
geometry1 = [Point(xy) for xy in zip(fc2017.Longitude, fc2017.Latitude)]
crs = seaNH_crs
fc2017_geo = GeoDataFrame(fc2017, crs = crs, geometry = geometry1)

geometry2 = [Point(xy) for xy in zip(fc2018.Longitude, fc2018.Latitude)]
crs = seaNH_crs
fc2018_geo = GeoDataFrame(fc2018, crs = crs, geometry = geometry2)

In [201]:
fc2017_geo.to_file("FC2017_FS_HOS")
fc2018_geo.to_file("FC2018_FS_HOS")